# Tracking Viewed Items Using Lists

Many websites provide users with a historical view of their browsing history. It could be
a list of recently viewed items from a product catalog, prior interactions with the
customer service team, or even a list of recently viewed blog posts - providing users with
historical information is a useful way to help them find items that were of interest in
the past. Regardless of the type of item, it's essential that your system can respond to
requests for the data as quickly as possible.

This chapter will look at at using the Redis list datatype to maintain historically viewed
items on a per user basis. Redis makes it very easy to write code to maintain this list as
well as provides fast access to the data when requested by the user. Our examples will
cover:

* Storing complex data in lists
* Specifying data storage conventions
* Identifying per-user lists
* Adding items to the recently viewed list
* Multiple ways of getting items from the list

At the end of the chapter, you should feel comfortable working with lists in Redis
and understand how to apply them to a variety of problems.

## Lists

Redis provides a list data structure which is used to represent ordered sequences of
elements.  Like other Redis data structures, every Redis list is associated with a name,
called a key, that is used to reference a particular list.  The maximum length of a list is 
2^32 - 1 elements, allowing for more than 4 billion elements per list.

4294967295, more than 4 billion of elements per list)

In Redis' terminology, the two ends of a list are referred to as the left and the right.  
The left is associated with the first element of the list and the right is associated with 
the last element of the list.

* * *

> **Tip**
>
> Redis' terminology for lists varies from some other languages and algorithm
> books.  Other sources use the term head where Redis uses left and the term tail 
> where Redis uses right.

* * *

The current version of Redis (3.2) provides seventeen different commands for
working with List.  Redis provides commands to:

* Determine the size of a list
* Add and remove items to a list
* Scan for items in a list
* Modify items in a list

One of the advantages that Redis provides when working with list data, is that
Redis allows you to modify your data on the database server, rather than having to
read the data from the database then write modifications back.

Most Redis list commands operate on the ends of a list or on a range of
elements.  Redis provides commands that operate on both the left and right ends
of the list, as well as commands that operate on a contiguous range of elements.
A few commands offer scanning or searching capabilities based on a binary comparison 
of elements.  Indexes into a Redis List are zero-based.

## Recently Viewed Items

In this chapter, we will use a JSON structure to capture properties about a previously
viewed blog article and store that data in Redis. Our JSON will contain three fields,
`date` (storing the date viewed), `title` (storing a string title of the blog post), and
`url` (permalink url to post).

Using our JSON to encode a blog post would look like:

In [ ]:
{
    "date": 1488573871,
    "title": "Unveiling the New Redis Enterprise Cloud UI",
    "url": "https://redislabs.com/blog/unveiling-new-redis-enterprise-cloud-ui/"
}


In our JSON data, both `title` and `url` will be encoded as strings.  The `date` will
be encoded as an integer number of seconds since the Unix Epoch.  The encoding
of date is picked to be compatible with the Python `time` package.

### Working with JSON in Redis

There are many different ways of storing JSON data in Redis: serialized strings, mapping
to hashes, natively using modules. For this example, we are going to store JSON in the
database as serialized strings.

The application code will be responsible for serializing JSON into a string prior to
inserting into Redis. The application will also need to deserialize JSON strings into a
in-memory form after fetching data items from Redis.

Most programming environments have multiple libraries for working with JSON data, in our
example code, we will use the Python `json` package.

### Working with JSON in Python

The `json` module is the standard JSON processing package for Python.  It is
part of the Python Standard Library and comes with all Python distributions.
The `json` package will provide all the functionality we need to use serialized
JSON strings with Redis.

In Python, JSON objects are generally represented using the built in dictionary
datatype.  Dictionaries can be marshaled (serialized) into strings using
the `json.dumps` function and strings can be unmarshaled (deserialized) into
dictionaries using the `json.loads` function.  If you aren't familiar with the
`json` package, take some time to study the following example:

In [ ]:
import json
# example Redis connection configuration stored using JSON, one way an app
# might store connection information
connection_cfg = {
    "prod": {
        "host": "prod-redis.mydomain.com",
        "port": 6379,
        "password": "secret",
        "db": 0
     },
     "staging": {
        "host": "staging-redis.mydomain.com",
        "port": 6379,
        "password": "notAsSecret",
        "db": 0    
     }
}

s = json.dumps(connection_cfg)
print ('Serialized JSON: """{}"""'.format(s))


If everything ran correctly, you should see a serialized JSON string that
represents the connection information structure originally stored in the
`connection_cfg` variable as the output from the code cell.  Because JSON
is designed to be human readable, the serialized string isn't that much
different than the original specification.

Take the output string from the serialization call  and paste it into the code 
below to see how it deserializes into a Python dictionary.  Because you are 
pasting a multi-line string, be sure to use the `"""` delimiter.

In [ ]:
import pprint
# Copy the serialized JK
serialized_cfg = """{"prod": {"host": "prod-redis.mydomain.com", "port": 6379, "password": "secret", "db": 0}, "staging": {"host": "staging-redis.mydomain.com", "port": 6379, "password": "notAsSecret", "db": 0}}"""

# Deserialize our configuration string and print it
cfg = json.loads(serialized_cfg)
pprint.pprint(cfg)


The output from this code cell should be a dictionary that looks nearly
identical to the `connection_cfg` above.  If the code doesn't run correctly,
check that you correctly pasted the serialized string from the section above 
and have used the multi-line quote delimiter `""`.

### Data Storage Conventions

Our historical items will be stored in the database using the List datatype.
One list will be stored for each user under the key `user:history:{id}`.  Each 
list will store up to ten of the most recently viewed items by that user.  We will 
standardize on the left of the list being the most recently viewed item and the right 
being the oldest item.  Each element of the list will be a serialized JSON string and
the list will be capped at ten items.  Ten is an arbitrary number based on our 
hypothetical feature specification.

To improve the clarity of our code, the Notebook environment includes several
utility functions, so you can focus on the Redis programming details.  For this
chapter, the main functions you may want to use from the `workshop` package are:

* `user_history_key` - generates the proper key from a user id
* `ViewedItem.to_serialized_json` - generates a serialized JSON string from a
ViewedItem
* `ViewedItem.from_serialized_json` - creates a ViewedItem object from a
serialized JSON string
* `clear_user_history` - removes a users history from Redis
* `add_history` - preloads a sample history into Redis for a user

We also provide a `ViewedItem` class to represent the application's in-memory
data structure.

## Adding an Item

When a user views an item on our example site, we need to add that item to the
recently viewed item list in Redis.  To accomplish this task, our code
will need to perform a few operations:

* Construct a `ViewedItem` instance with the proper data
* Serialize the `ViewedItem` into a string
* Insert the string into the database (checking for duplicates)
* Ensure our lists has no more than 10 elements

This can be accomplished with a few lines of code and three Redis commands:
LREM, LPUSH and LTRIM.

The LREM (**L**ist **REM**ove) command removes the specified number of copies of
an item from a list.  This command is used to remove the viewed item from the
list if it is already there.  The LPUSH (**L**eft **PUSH**) prepends an item to
the beginning of a Redis List, so we use this command to add our most recently
viewed item to the list.  Finally the LTRIM (**L**ist **TRIM**) command trims
the list to the specified range of elements, which we use to ensure our list is
only the last ten viewed items.

In [ ]:
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379,
    "charset" : "utf-8",
    "decode_responses" : True
}

r = redis.StrictRedis(**config)

In [ ]:
class ViewedItem(object):

    def __init__(self, **kwargs):
        self.date = None
        self.url = None
        self.title = None

        for key in kwargs:
            setattr(self, key, kwargs[key])
            
    def __str__(self):
        return "%s: %s" % (self.id, self.name)

    def __repr__(self):
        from pprint import pformat
        return pformat(vars(self))


    def to_serialized_json(self):
        return json.dumps(self.__dict__)

    @staticmethod
    def from_serialized_json(s):
        return ViewedItem(**json.loads(s))

In [ ]:
import time
def user_history_key(user_id):
    "Generates the appropriate Redis key for the last viewed items of the supplied user_id"

    return "user:history:" + str(user_id)

def add_viewed_item(r, user_id, item):
    "Stores the most recently viewed item by the user into the Redis database"
    
    # serialize our item
    serialized_json = item.to_serialized_json()
    
    # get our key
    key = user_history_key(user_id)
    
    # remove item from list if it is already there
    removedCnt = r.lrem(key, 1, serialized_json)
    print("number of elements removed " + str(removedCnt))
    
    # store in Redis, returns length of list, which we ignore
    r.lpush(key, serialized_json)

    # trim the list (from the left) to 10 items
    r.ltrim(key, 0, 9)

# construct our currently viewed item and serialize it
item = ViewedItem(date=int(time.time()),
                  url='https://redislabs.com/blog/unveiling-new-redis-enterprise-cloud-ui/',
                  title='Unveiling the New Redis Enterprise Cloud UI')

add_viewed_item(r, 3001, item)  


Using the `redisinsight` browse capability, we can display the state
of the Redis database after trying the `add_viewed_item` function.  You should
see a new key in the database for user 3001 that references a list of items.



## Accessing the Most Recently Viewed Posts

Now that we are storing information about historical views, we need code to
access that information. Redis provides a wide range of functions for accessing 
list data.  We are going to start with a function to retrieve the most
recently viewed item.

### Fetching Data

One way to implement our function is to:

* Retrieve an item from the left of the list
* Construct a `ViewedItem` object from the serialized string

Redis provides different functions we could use to get the data item from the left of the
list. In this sample, we are going to use the Redis LPOP (**L**eft **POP**) command, which
removes an item from the left of the list and returns it to the caller. Once we have
retrieved the item data from Redis, we use the static method
`ViewItem.from_serialized_json` to construct our object representation of the data.

*Run the example below by selecting the code cell, then using SHIFT + RETURN to execute the code.*

In [ ]:
sample_history_data = [
    {
        "date": 1488573871,
         "title": "Unveiling the New Redis Enterprise Cloud UI",
         "url": "https://redislabs.com/blog/unveiling-new-redis-enterprise-cloud-ui/"
    },
    {
        "date": 1488573971,
         "title": "First Ever Redis Modules Hackaton",
         "url": "https://redislabs.com/blog/first-ever-redis-modules-hackathon/"
    },
    {
        "date": 1488574071,
         "title": "Redis Pack v4.4 Release",
         "url": "https://redislabs.com/blog/redis-pack-v4-4-release/"
    },
    {
        "date": 1488574271,
         "title": "Count Min Sketch: The Art and Science of Estimating Stuff",
         "url": "https://redislabs.com/blog/count-min-sketch-the-art-and-science-of-estimating-stuff/"
    },
    {
        "date": 1488574471,
         "title": "Redis Cloud Integrates with Databricks Spark",
         "url": "https://redislabs.com/blog/redis-cloud-integrates-with-databricks-spark/"
    },
    {
        "date": 1488574571,
         "title": "ZeroBrane Studio Plugin for Redis Lua Scripts",
         "url": "https://redislabs.com/blog/zerobrane-studio-plugin-for-redis-lua-scripts/"
    },
    {
        "date": 1488574771,
         "title": "Connecting Spark and Redis: A Detailed Look",
         "url": "https://redislabs.com/blog/connecting-spark-and-redis-a-detailed-look/"
    },
    {
        "date": 1488574871,
         "title": "Top Redis Headaches for DevOps - Client Buffers",
         "url": "https://redislabs.com/blog/top-redis-headaches-for-devops-client-buffers/"
    },
    {
        "date": 1488575071,
         "title": "5^H 6^H 7 Methods for Tracing and Debuging Redis Lua Scripts",
         "url": "https://redislabs.com/blog/5-6-7-methods-for-tracing-and-debugging-redis-lua-scripts/"
    },
    {
        "date": 1488575871,
         "title": "Redis is Beautiful: A Vizualization of Redis Commands",
         "url": "https://redislabs.com/blog/red-is-beautiful-a-visualization-of-redis-commands/"
    }
]

def add_history(r, user_id):
    "Adds some user history data to Redis for a given user"

    sample_history_data.sort(key=lambda hist:  hist['date'])

    values = [ ]
    for hist in sample_history_data:

        item = ViewedItem(**hist)
        serialized_json = item.to_serialized_json()
        values.append(serialized_json)


    r.lpush(user_history_key(user_id), *values)
    
def get_most_recent_for_user(r, user_id):
    "Returns the most recent viewed item for a user"
    
    key = user_history_key(user_id)
    item_json = r.lindex(key,0)
    # can use lpop but that removes the element
    if item_json is not None:
        item = ViewedItem.from_serialized_json(item_json)
        return item
    else:
        return None

# utility functions to set up our database
# workshop.lists.clear_user_history(r, 1001)
# workshop.lists.clear_user_history(r, 3001)
add_history(r, 3001)


# simulate for user 1001
item = get_most_recent_for_user(r, 1001)
pprint.pprint("User Id: 1001") 
pprint.pprint(item)

# simulate for user 3001
item = get_most_recent_for_user(r, 3001)
pprint.pprint("User Id: 3001")
pprint.pprint(item)


If your code executed properly, you should see a `None` result returned for user 1001 and
an item, *Redis is Beautiful: A Vizualization of Redis Commands*, for user 3001. If you
got different results, keep in mind that all of the Notebook cells run against the same
Redis database. You may need to use some of the various utility functions to reset your
database's state.  Use redisinsights to see new value.

LPOP is one of the first ways developers learn to manipulate a list, but depending on how
the historical view functionality should work it may not be the right function to use for
this feature. LPOP performs a destructive modification of the list, changing our
historical view, which may not be what the users want.

### Getting The Oldest Viewed Item

An alternative implementation is to access the oldest viewed item.  For
this code, we would need a function that:

* Retrieve an item from the right of the list
* Construct a `ViewedItem` object from the serialized string

Notice, this procedure is nearly identical to the procedure for finding the newest item,
only we need to take the item from the right instead of the left.  This can be
accomplished by changing one Redis command.  In this case we need to change the 
LPOP to an RPOP command.

In [ ]:
def clear_user_history(r, user_id):
    "Clears the previous viewed history for a user from Redis"
    r.delete(user_history_key(user_id))
    
def get_least_recent_for_user(r, user_id):
    "Returns the least recent viewed item for a user"
    
    key = user_history_key(user_id)
    #  can use rpop but that removes the element
    item_json = r.lindex(key,-1)
    if item_json is not None:
        item = ViewedItem.from_serialized_json(item_json)
        return item
    else:
        return None
    
# utility functions to set up our database
clear_user_history(r, 1001)
clear_user_history(r, 3001)
add_history(r, 3001)

# simulate for user 1001
item = get_least_recent_for_user(r, 1001)
pprint.pprint("User Id: 1001") 
pprint.pprint(item)

# simulate for user 3001
item = get_least_recent_for_user(r, 3001)
pprint.pprint("User Id: 3001")
pprint.pprint(item)


Notice how in both examples, Redis didn't return an error even though we never created the
list for user 1001. Redis often operates like a scripting language, trying to return
sensible defaults for missing data instead of returning errors that require additional
logic to process.

Like the previous example using LPOP, this example destructively modifies our viewing
history. Most viewing histories operate differently, the historical view is only changed
when you look at a new item. Again, Redis makes it very easy to implement this
functionality. The LINDEX command takes a list key and an index and returns the element at
the index without modifying the list. In the cell below reimplement the
`get_most_recent_for_user` function to return the most recently viewed item without
modifying the list.

### Getting All Items

Getting only one result or looping over all the historical items and getting one element
at a time, is not going to make for a satisfying feature. Eventually, we will need to get
all of the items from Redis in a single call.

Using the LRANGE command, you can read a range of items from a list without modifying the
stored list. The developer specifies a start and end position to the LRANGE command, which
returns the *inclusive* range specified by those two positions relative to the **left**
end of the list. As an example, given the list [0, 1, 2, 3, 4, 5], the range 1, 3 would
return the list [1, 2, 3]. Negative indexes can be used to reference from the right of the
list.

The LRANGE command takes a start and end position and returns the inclusive range
specified by those two positions from the **left**. As an example, if I had the list [0,
1, 2, 3, 4, 5], the range 1 to 3 would return the list [1, 2, 3]. Negative indexes can be
used to reference from the right (end) of a list. Specifying the range -3,-1 to LRANGE
would return [3, 4, 5].

* * *

> **Tip**
>
> Although negative indexes in Redis reference from the right of the list,
> ranges still have to define a contiguous block of elements that run from left to 
> right.  So the range -3, -1 will return results, the range -1, -3 returns an empty list.
>

* * *

Like most Redis commands for reading data, LRANGE tries to return a sensible
result for missing data. The example code below, shows how you can use LRANGE 
to fetch a users entire history list.

In [ ]:
def get_recently_viewed_items(r, user_id):
    "Returns a list of the most recently viewed items of the given user"
    
    key = user_history_key(user_id)
    
    db_results = r.lrange(key, 0, -1)
    results = []
    for db_result in db_results:
        results.append(ViewedItem.from_serialized_json(db_result))
    
    return results

# utility functions to set up our database
clear_user_history(r, 3001)
add_history(r, 3001)

pprint.pprint(get_recently_viewed_items(r, 3001))


If your example runs as expected, you should see a printout of the list of
`ViewedItems` representing the user's data.

## Scaling 

As your data set changes, you may need to reconsider the implementation 
used in this particular example.  For simplicity, the example code stores
a copy of the complete historically viewed item in each element - redundantly
storing both the `title` and `url` properties across multiple users.  While
the amount of memory that an user might be reasonable, the cost of storing that
redundant information may become prohibitive as your user population grows.

To support a large data set, you might consider using a more compact encoding
than JSON and storing the date along with a key, such as an integer item id, 
that can be used to look up the other details about the item. 

## Conclusion

The Redis List datatype is a flexible way of storing ordered sequences of data.
In our example, we use it to maintain a list of historically viewed items, but
the underlying data structure can be used for a variety of features.  You could
use lists to maintain a user's news feed, implement a widget that displays a
series of tweets, or even order user votes over time.

Redis lists can be used for a variety of other tasks.  Many developers - and
some libraries - use lists to maintain work and task queues, other developers
have used lists to exchange messages between processes.  More details regarding
lists can be viewed on the [List Commands](https://redis.io/commands#list) page
on [Redis.io](https://redis.io).